# Homework 4

### Fanchen Zeng (fz2241)

***

> 1. Give a count of all videos currently out.

> 2. Make a list of all customer names who have videos out and how much they owe.

> 3. Give the most popular actors by store location.

> 4. Using a $1 per day late fee. Find out which users owe the most assuming all rentals are a week long.

> 5. What hour of the day to people rent the most ?

> 6. Which store is more profitable, assuming all movies cost $15 per inventory item to purchase.

> 7. Given a movie name, find out movies to recommend, based on the most popular movies rented by people who rented *that* movie.

***

## Question 1

### Give a count of all videos currently out
```
SELECT count(rental_id)
FROM rental
WHERE return_date IS NULL;
```

**Return**: 183

***

## Question 2

### Make a list of all customer names who have videos out and how much they owe

column 'owe': how many DVDs the customer did not return
```
SELECT pp.customer_id,
       c.first_name,
       c.last_name,
       pp.owe
FROM
  (SELECT c.customer_id,
          count(c.customer_id) owe
   FROM customer c
   INNER JOIN rental r ON r.customer_id = c.customer_id
   WHERE return_date IS NULL
   GROUP BY 1) pp
INNER JOIN customer c ON c.customer_id = pp.customer_id
ORDER BY 4 DESC;
```

**Return**

<div style='float: center'>
  <img style='width: 600px' src="picture/2.png"></img>
</div>

***

## Question 3

### Give the most popular actors by store location ( I use city_id rather than store_id since there are only two shops in the table but have many cities

**Hint**: Firstly, I query the amount of films every actor appears in every city. Then I find the maximum amount for every city. So I get for every city, which actors appear most times.

```
SELECT pppp.city_id,
       cit.city city_name,
       pppp.actor_id,
       ac.first_name,
       ac.last_name,
       pppp.max_count appear_times_in_one_city
FROM
  (SELECT ppp.city_id,
          ppp.actor_id,
          pp.max_count
   FROM
     (SELECT p.city_id,
             max(p.number) max_count
      FROM
        (SELECT ci.city_id,
                fa.actor_id,
                count(fa.actor_id) number
         FROM film_actor AS fa
         INNER JOIN film f ON f.film_id = fa.film_id
         INNER JOIN actor a ON a.actor_id = fa.actor_id
         INNER JOIN inventory i ON i.film_id = f.film_id
         INNER JOIN rental r ON r.inventory_id = i.inventory_id
         INNER JOIN customer c ON c.customer_id = r.customer_id
         INNER JOIN address ad ON ad.address_id = c.address_id
         INNER JOIN city ci ON ci.city_id = ad.city_id
         GROUP BY ci.city_id,
                  fa.actor_id) p
      GROUP BY p.city_id) pp
   INNER JOIN
     (SELECT ci.city_id,
             fa.actor_id,
             count(fa.actor_id) number
      FROM film_actor AS fa
      INNER JOIN film f ON f.film_id = fa.film_id
      INNER JOIN actor a ON a.actor_id = fa.actor_id
      INNER JOIN inventory i ON i.film_id = f.film_id
      INNER JOIN rental r ON r.inventory_id = i.inventory_id
      INNER JOIN customer c ON c.customer_id = r.customer_id
      INNER JOIN address ad ON ad.address_id = c.address_id
      INNER JOIN city ci ON ci.city_id = ad.city_id
      GROUP BY ci.city_id,
               fa.actor_id) ppp ON pp.city_id = ppp.city_id
   AND pp.max_count = ppp.number) pppp
INNER JOIN actor ac ON pppp.actor_id = ac.actor_id
INNER JOIN city cit ON cit.city_id = pppp.city_id
```

**Return**

<div style='float: center'>
  <img style='width: 600px' src="picture/3.png"></img>
</div>

***

## Question 4

### What hour of the day to people rent the most ?

```
SELECT c.customer_id,
       c.first_name,
       c.last_name,
       date(r.return_date) - date(r.rental_date) - 7 AS late_fee
FROM rental r
INNER JOIN customer c ON c.customer_id = r.customer_id
WHERE date(r.return_date) - date(r.rental_date) - 7 >= 0
ORDER BY 4 DESC;
```

**Return**: 15:00

<div style='float: center'>
  <img style='width: 600px' src="picture/4.png"></img>
</div>

***

## Question 5

### What hour of the day to people rent the most ?

```
SELECT extract(hour
               FROM rental_date) AS rental_hour,
       count(extract(hour
                     FROM rental_date))
FROM rental
GROUP BY rental_hour
ORDER BY 2 DESC;
```

**Return**: 15:00

<div style='float: center'>
  <img style='width: 600px' src="picture/5.png"></img>
</div>

***

## Question 6

### Which store is more profitable, assuming all movies cost 15 dollar per inventory item to purchase.

**Hint**: Firstly, inner join table of 'Rental' and 'Payment' provide the money shops earn (Although the table 'Payment' doesn't contain all records of rental_id. Secondly, table 'inventory' give us the total amount of inventories in each shop and multiply 15 we get the cost of each shop pay for inventories. Finally subtract earn and cost, we get profit.

```
SELECT ii.store_id, ppp.earn, ii.cost, (ppp.earn - ii.cost) profit
FROM
  (SELECT store_id,
          count(inventory_id) * 15. AS cost
   FROM inventory
   GROUP BY 1) ii
INNER JOIN
  (SELECT i.store_id,
          sum(ss) earn
   FROM
     (SELECT r.inventory_id,
             sum(p.amount) ss
      FROM rental r
      INNER JOIN payment p ON r.rental_id = p.rental_id
      GROUP BY 1) pp
   INNER JOIN inventory i ON i.inventory_id = pp.inventory_id
   GROUP BY 1) ppp ON ppp.store_id = ii.store_id
ORDER BY 4 DESC

```

**Return**: store_id = 1

<div style='float: center'>
  <img style='width: 600px' src="picture/6.png"></img>
</div>

***

## Question 7

### Given a movie name, find out movies to recommend, based on the most popular movies rented by people who rented *that* movie.

**I delete the first row which is *that* moive('Freaky Pocus')**

**Hint**: Firstly, I find which customer_id watch the film 'Freaky Pocus'. Then use these customer_id to find which film_id they watched. Finally, count the amount of film_id every related customer watched and delete the first row, which is the film 'Freaky Pocus'.

```
SELECT *
FROM
  (SELECT ROW_NUMBER() OVER(
                            ORDER BY count(pp.title) DESC) AS ROW,
          pp.title,
          count(pp.title)
   FROM
     (SELECT r.customer_id
      FROM film f
      INNER JOIN inventory i ON f.film_id = i.film_id
      INNER JOIN rental r ON r.inventory_id = i.inventory_id
      WHERE f.title LIKE 'Freaky Pocus') p
   INNER JOIN
     (SELECT r.customer_id,
             f.title
      FROM film f
      INNER JOIN inventory i ON f.film_id = i.film_id
      INNER JOIN rental r ON r.inventory_id = i.inventory_id) pp ON p.customer_id = pp.customer_id
   GROUP BY 2
   ORDER BY 3 DESC) ppp
WHERE ROW >= 2
```

**Given**: Freaky Pocus

**Return**: Voyage Legally, Bonnie Holocaust and Masked Bubble

<div style='float: center'>
  <img style='width: 600px' src="picture/7.png"></img>
</div>

***